In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys, os
sys.path.append(os.path.abspath("../.."))
from configs import GOOGLE_APPLICATION_CREDENTIALS,GCS_BUCKET_NAME,GCS_PROJECT_ID
from google.cloud import bigquery
from src.utils.io_utils import upload_to_bigquery
from clean_utils import *

In [2]:
client = bigquery.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
table_id = f"{GCS_PROJECT_ID}.{GCS_BUCKET_NAME}.data_cleaned"

In [3]:
query = """SELECT *
FROM `khangtestdbt.xecupredict.oto_com` """
data_oto = client.query(query).to_dataframe()
data_oto.head(1)

,Name,Price,Năm_SX,Nhiên_liệu,Kiểu_dáng,Tình_trạng,Km_đã_đi,Hộp_số,Xuất_xứ,Tỉnh_thành,Số_tiền_vay,Thời_gian_vay,Lãi_xuất,Loại_hình,Giá_trị_xe_muốn_mua
0,Toyota Raize 2024,510 triệu,2024,Máy xăng,None,Xe cũ,16.913 km,Số tự động,Nhập khẩu,Tp.HCM,VNĐ,ThángNăm,ThángNăm,"Trả góp đều hàng thángTrả góp đều, lãi tính tr...",VNĐ


In [4]:
df_oto = data_oto.copy()

In [5]:
df_oto.columns

Index(['Name', 'Price', 'Năm_SX', 'Nhiên_liệu', 'Kiểu_dáng', 'Tình_trạng',
       'Km_đã_đi', 'Hộp_số', 'Xuất_xứ', 'Tỉnh_thành', 'Số_tiền_vay',
       'Thời_gian_vay', 'Lãi_xuất', 'Loại_hình', 'Giá_trị_xe_muốn_mua'],
      dtype='object')

In [6]:
df_oto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Name                 1479 non-null   object
 1   Price                1479 non-null   object
 2   Năm_SX               1479 non-null   object
 3   Nhiên_liệu           1479 non-null   object
 4   Kiểu_dáng            1380 non-null   object
 5   Tình_trạng           1479 non-null   object
 6   Km_đã_đi             1475 non-null   object
 7   Hộp_số               1474 non-null   object
 8   Xuất_xứ              1460 non-null   object
 9   Tỉnh_thành           1479 non-null   object
 10  Số_tiền_vay          1479 non-null   object
 11  Thời_gian_vay        1479 non-null   object
 12  Lãi_xuất             1479 non-null   object
 13  Loại_hình            1479 non-null   object
 14  Giá_trị_xe_muốn_mua  1479 non-null   object
dtypes: object(15)
memory usage: 173.6+ KB


In [7]:
df_oto.describe(include='all').T

,count,unique,top,freq
Name,1479,1100,Mercedes-Benz GLC 300 4Matic 2023,9
Price,1479,558,465 triệu,20
Năm_SX,1479,27,2022,217
Nhiên_liệu,1479,4,Máy xăng,1237
Kiểu_dáng,1380,13,SUV,528
Tình_trạng,1479,1,Xe cũ,1479
Km_đã_đi,1475,353,100.000 km,49
Hộp_số,1474,3,Số tự động,1293
Xuất_xứ,1460,2,Nhập khẩu,754
Tỉnh_thành,1479,147,Tp.HCM,173


1.name

In [8]:
df_oto["name"] = df_oto["Name"].apply(clean_name)
df_oto.drop(columns=["Name"], inplace=True)
df_oto.head(5)

,Price,Năm_SX,Nhiên_liệu,Kiểu_dáng,Tình_trạng,Km_đã_đi,Hộp_số,Xuất_xứ,Tỉnh_thành,Số_tiền_vay,Thời_gian_vay,Lãi_xuất,Loại_hình,Giá_trị_xe_muốn_mua,name
0,510 triệu,2024,Máy xăng,None,Xe cũ,16.913 km,Số tự động,Nhập khẩu,Tp.HCM,VNĐ,ThángNăm,ThángNăm,"Trả góp đều hàng thángTrả góp đều, lãi tính tr...",VNĐ,Toyota Raize 2024
1,280 triệu,2007,Máy xăng,SUV,Xe cũ,130.000 km,Số tự động,Nhập khẩu,Hà Nội,VNĐ,ThángNăm,ThángNăm,"Trả góp đều hàng thángTrả góp đều, lãi tính tr...",VNĐ,Toyota RAV4 LE 2007
2,475 triệu,2020,Máy xăng,SUV,Xe cũ,80.000 km,Số tự động,Nhập khẩu,Cầu Giấy - Hà Nội,VNĐ,ThángNăm,ThángNăm,"Trả góp đều hàng thángTrả góp đều, lãi tính tr...",VNĐ,Toyota Rush S 1 5 AT 2020
3,490 triệu,2021,Máy xăng,SUV,Xe cũ,44.000 km,Số tự động,Nhập khẩu,Thanh Xuân - Hà Nội,VNĐ,ThángNăm,ThángNăm,"Trả góp đều hàng thángTrả góp đều, lãi tính tr...",VNĐ,Toyota Rush S 1 5 AT 2021
4,480 triệu,2022,Máy xăng,SUV,Xe cũ,30.000 km,Số tự động,Nhập khẩu,Hà Nội,VNĐ,ThángNăm,ThángNăm,"Trả góp đều hàng thángTrả góp đều, lãi tính tr...",VNĐ,Toyota Rush 1 5 AT 2022


2.price

In [9]:
df_oto["Price"].isna().sum()

np.int64(1)

In [10]:
def clean_price(x):
    if pd.isna(x):
        return None
    
    x = str(x).lower().strip()

    if x in ["", "none", "nan", "giá liên hệ", "contact", "-"]:
        return None

    x = x.replace("triệu", "triệu").replace("trieuj", "triệu")
    x = x.replace("ty", "tỷ").replace("ti", "tỷ")

    x = x.replace("tỉ", "tỷ")

    x = re.sub(r"\s+", " ", x)

    if "triệu" not in x and "tỷ" not in x:
        digits = re.sub(r"[^\d]", "", x)
        return int(digits) if digits.isdigit() else None

    x = x.replace(",", ".")  

    match = re.match(r"(\d*\.?\d*)\s*tỷ\s*(\d*\.?\d*)?\s*triệu?", x)
    if match:
        ty = float(match.group(1)) if match.group(1) else 0
        trieu = float(match.group(2)) if match.group(2) else 0
        return int(ty * 1_000_000_000 + trieu * 1_000_000)

    if "tỷ" in x:
        num = re.findall(r"[\d\.]+", x)
        if num:
            return int(float(num[0]) * 1_000_000_000)
        return None

    if "triệu" in x:
        num = re.findall(r"[\d\.]+", x)
        if num:
            return int(float(num[0]) * 1_000_000)
        return None

    return None


In [11]:
df_oto["price"] = df_oto["Price"].apply(clean_price)
df_oto.drop(columns=["Price"], inplace=True)
df_oto.sample(5)

,Năm_SX,Nhiên_liệu,Kiểu_dáng,Tình_trạng,Km_đã_đi,Hộp_số,Xuất_xứ,Tỉnh_thành,Số_tiền_vay,Thời_gian_vay,Lãi_xuất,Loại_hình,Giá_trị_xe_muốn_mua,name,price
1439,2021,Máy xăng,Sedan,Xe cũ,75.000 km,Số tự động,Trong nước,Thanh Xuân - Hà Nội,VNĐ,ThángNăm,ThángNăm,"Trả góp đều hàng thángTrả góp đều, lãi tính tr...",VNĐ,Kia Cerato 1 6 AT Luxury 2021,4.750000e+08
441,2014,Máy xăng,MPV,Xe cũ,125.911 km,Số tự động,Nhập khẩu,Tp.HCM,VNĐ,ThángNăm,ThángNăm,"Trả góp đều hàng thángTrả góp đều, lãi tính tr...",VNĐ,Toyota Innova 2 0G 2014,3.400000e+08
337,2020,Máy xăng,Sedan,Xe cũ,50.000 km,Số tự động,Nhập khẩu,Thanh Xuân - Hà Nội,VNĐ,ThángNăm,ThángNăm,"Trả góp đều hàng thángTrả góp đều, lãi tính tr...",VNĐ,Toyota Camry 2 0Q 2020,8.800000e+08
563,2019,Máy xăng,SUV,Xe cũ,42.000 km,Số tự động,Nhập khẩu,Tân Bình - Tp.HCM,VNĐ,ThángNăm,ThángNăm,"Trả góp đều hàng thángTrả góp đều, lãi tính tr...",VNĐ,Porsche Macan 2 0 2019,2.279000e+09
97,2019,Máy xăng,SUV,Xe cũ,140.000 km,Số tự động,Trong nước,Hà Nội,VNĐ,ThángNăm,ThángNăm,"Trả góp đều hàng thángTrả góp đều, lãi tính tr...",VNĐ,VinFast LUX SA2 0 Cao cấp 2019,5.550000e+08


In [12]:
df_oto["price"].isna().sum()

np.int64(1)

In [13]:
df_oto = df_oto[df_oto["price"] > 50000000]

3.brand

In [14]:
df_oto["brand"] = df_oto["name"].str.strip().str.split().str[0]

In [15]:
df_oto.brand.value_counts()

brand
Toyota        249
Hyundai       166
Mercedes      160
Kia           157
Ford          134
Mitsubishi     82
VinFast        75
Mazda          73
Honda          62
Lexus          46
MG             36
BMW            36
Chevrolet      29
Volvo          25
Porsche        25
Peugeot        23
Nissan         21
Suzuki         20
Land           11
Volkswagen     10
Mini            7
Audi            6
Acura           3
Daewoo          3
Daihatsu        2
Subaru          2
Rolls           2
Thaco           1
Smart           1
Samsung         1
Skoda           1
SRM             1
Bentley         1
Maserati        1
GAC             1
Citroen         1
Dongben         1
Jaguar          1
KenBo           1
Name: count, dtype: int64

In [16]:
df_oto["brand"].isna().sum()

np.int64(0)

In [17]:
df_oto["brand"] = df_oto["brand"].str.lower()

4.year

In [18]:
df_oto["Năm_SX"].head(5)

0    2024
1    2007
2    2020
3    2021
4    2022
Name: Năm_SX, dtype: object

In [19]:
df_oto["age"] = 2025 - df_oto["Năm_SX"].astype(int)
df_oto.drop(columns=["Năm_SX"], inplace=True)
df_oto.head(1)

,Nhiên_liệu,Kiểu_dáng,Tình_trạng,Km_đã_đi,Hộp_số,Xuất_xứ,Tỉnh_thành,Số_tiền_vay,Thời_gian_vay,Lãi_xuất,Loại_hình,Giá_trị_xe_muốn_mua,name,price,brand,age
0,Máy xăng,None,Xe cũ,16.913 km,Số tự động,Nhập khẩu,Tp.HCM,VNĐ,ThángNăm,ThángNăm,"Trả góp đều hàng thángTrả góp đều, lãi tính tr...",VNĐ,Toyota Raize 2024,510000000.0,toyota,1


In [20]:
df_oto["age"].isna().sum()

np.int64(0)

5.status

In [21]:
df_oto["Tình_trạng"].unique()

array(['Xe cũ'], dtype=object)

In [22]:
df_oto.drop(columns=["Tình_trạng"], inplace=True)

6.km

In [23]:
df_oto["Km_đã_đi"].head(5)

0     16.913 km
1    130.000 km
2     80.000 km
3     44.000 km
4     30.000 km
Name: Km_đã_đi, dtype: object

In [24]:
df_oto.rename(columns={"Km_đã_đi":"km"}, inplace=True)

In [25]:
df_oto["km"].isna().sum()

np.int64(4)

In [26]:
def cv_km(x):
    if pd.isna(x):        
        return None
    x = str(x).lower().strip()
    if x in ["", "-", "none"]:    
        return None
    x = (
        x.replace("km", "")
         .replace(".", "")
         .replace(",", "")
         .strip()
    )
    if x.isdigit():
        return int(x)
    return None          


In [27]:
df_oto["km"] = df_oto["km"].apply(cv_km)

In [28]:
df_oto = df_oto[df_oto["km"] > 100]

In [29]:
df_oto["km"].isna().sum()

np.int64(0)

7.origin

In [30]:
df_oto["Xuất_xứ"].isna().sum()

np.int64(16)

In [31]:
df_oto["Xuất_xứ"].value_counts()

Xuất_xứ
Nhập khẩu     729
Trong nước    677
Name: count, dtype: int64

In [32]:
df_oto["Xuất_xứ"].describe()

count          1406
unique            2
top       Nhập khẩu
freq            729
Name: Xuất_xứ, dtype: object

In [33]:
df_oto = df_oto.rename(columns={"Xuất_xứ": "origin"})

In [34]:
df_oto["origin"] = df_oto["origin"].replace({
    "Trong nước": "trong nước",
    "Nhập khẩu": "nhập khẩu"
})


8.body

In [35]:
df_oto["Kiểu_dáng"].unique()

array([None, 'SUV', 'MPV', 'Van/Minivan', 'Crossover', 'Sedan',
       'Hatchback', 'Special Purpose', 'Bán tải', 'Coupe', 'Convertible',
       'Sport Car', 'Xe tải', 'Minibus'], dtype=object)

In [36]:
df_oto = df_oto.rename(columns={"Kiểu_dáng": "body"})
df_oto["body"] = df_oto["body"].str.lower().str.strip()

In [37]:
df_oto["body"].isna().sum()

np.int64(98)

9.fuel

In [38]:
df_oto["Nhiên_liệu"].unique()

array(['Máy xăng', 'Điện', 'Hybrid', 'Máy dầu'], dtype=object)

In [39]:
df_oto = df_oto.rename(columns={"Nhiên_liệu": "fuel"})
df_oto["fuel"] = df_oto["fuel"].apply(clean_fuel)

In [40]:
df_oto["fuel"].unique()

array([None, 'Điện', 'Hybrid'], dtype=object)

In [41]:
df_oto["fuel"].isna().sum()

np.int64(1366)

In [42]:
df_oto["fuel"].value_counts()

fuel
Điện      36
Hybrid    20
Name: count, dtype: int64

10.updata

In [43]:
df_oto = df_oto[['fuel', 'body', 'km', 'origin', 'name','price', 'brand', 'age']]

In [44]:
df_oto.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1422 entries, 0 to 1479
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   fuel    56 non-null     object 
 1   body    1324 non-null   object 
 2   km      1422 non-null   float64
 3   origin  1406 non-null   object 
 4   name    1422 non-null   object 
 5   price   1422 non-null   float64
 6   brand   1422 non-null   object 
 7   age     1422 non-null   int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 100.0+ KB


In [45]:
upload_to_bigquery(df_oto, table_id=table_id, if_exists="append" )

✅ Uploaded 1422 rows to khangtestdbt.xecupredict.data_cleaned


In [46]:
# end